In [1]:
# Import neccessary libraries
import subprocess
import json
import os
from web3 import Web3
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy
from eth_account import Account
from constants import *
from dotenv import load_dotenv
load_dotenv()

# Import Mnemonic key from an environment variable
mnemonic = os.getenv('MNEMONIC', 'lucky ozone slender piano gather virus genuine couple birth pilot emerge narrow stay bread bring')

# Create functions that derives the wallet keys 
def derive_wallets(mnemonic, coin, numderive):
    command = f'./derive -g --mnemonic="{mnemonic}" --cols=all --coin={coin} --numderive={numderive} --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    keys = json.loads(output)
    return keys

# Store three constants in 'coins' variable 
coins = {
    ETH: derive_wallets(mnemonic, ETH, 3),
    BTCTEST: derive_wallets(mnemonic, BTCTEST, 3),
    BTC: derive_wallets(mnemonic, BTC, 3)
} 

w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

private_key = os.getenv("PRIVATE_KEY")

# Initiate the private keys for ETH and BTCTEST 
keys = {}
for coin in coins:
    keys[coin]= derive_wallets(mnemonic, coin, numderive=3)

eth_priv_key = keys['eth'][0]['privkey']
btc_test_priv_key = keys['btc-test'][0]['privkey']

print(f" ETH Private Key: {eth_priv_key}")
print(f" BTC Test Private Key: {btc_test_priv_key}")

Python-dotenv could not parse statement starting at line 1


 ETH Private Key: 0x290b8cc766fb866206228d5ac88baf8d315607ed612fc98b92ed36029b9faee1
 BTC Test Private Key: cRX6LxpH3n8LcFU1dM9LgjezMwMfgwEQFfrUbDLQWZarHkaAEtWT


In [2]:
print(json.dumps(keys, indent=4, sort_keys=True))

{
    "btc": [
        {
            "address": "1Dhm9S73e1TiQ4E1cyMSckUTN3fBC76Tb2",
            "index": 0,
            "path": "m/44'/0'/0'/0/0",
            "privkey": "L587w8hjD1cyvyLTgnzdhLKBwTtcezmXWroca1N4mvyNdWjn4SS7",
            "pubkey": "021639a0c4bdf14239e8e485a3b22b9fc1194f8044819d786e8b537d1baf729986",
            "pubkeyhash": "8b56ffa3f84481a7a10ab606ca9e415cecc6e733",
            "xprv": "xprvA2SzFeDUQLWeczRRqtwpnXtvmuxH2oveis2SPj99dtgsY9BRaKikUyavhs4npQuf9efgcyBfeuG61jQujzXiGNcdGX1bVmbaKEUavAjg28w",
            "xpub": "xpub6FSLf9kNEi4wqUVtwvUq9fqfKwnmSGeW65x3C7YmCEDrQwWa7s312muQZ6UVBcs8MqtyNKBucZjUikxoZX9CgXvLx1M2Gn6H3LQmdFFv8bg"
        },
        {
            "address": "1BNZgKsU1LnzYrvRTufSeEErUzaSdukN5m",
            "index": 1,
            "path": "m/44'/0'/0'/0/1",
            "privkey": "L4DNQ23fEcdZCvZwPWRWHm7Uro4EfwxZa38oLqGW4yEr2Q6pnSwQ",
            "pubkey": "02ce369c30c111c32e695b508bd09260dec8bc57791721eb882e4470865b82741a",
            "pubkeyhash":

In [3]:
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [4]:
# Create function that contains all metadata needed for transaction
def create_tx(coin, account, to, amount):
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas(
        {"from": eth_account.address, "to": recipient, "value": amount}
    )
        return {
            "from": eth_account.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(eth_account.address)}
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])

In [5]:
# Create function to sign the transaction
eth_account = priv_key_to_account(ETH, derive_wallets(mnemonic, ETH, 6)[0]['privkey'])
btc_test_account = priv_key_to_account(BTCTEST,btc_test_priv_key)

def send_tx(coin, account, to, amount):
    tx = create_tx(coin, account, recipient, amount)
    
    if coin == ETH:
        signed_tx = eth_account.sign_transaction(tx)
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        print(result.hex())
        return w3.eth.sendRawTransaction(signed.rawTransaction)
    elif coin == BTCTEST:
        tx_btc_test = create_tx(coin, account, recipient, amount)
        signed_txn = account.sign_transaction(txn)
        print(signed_txn)
        return NetworkAPI.broadcast_tx_testnet(signed)

In [10]:
btc_test_account = priv_key_to_account(BTCTEST,btc_test_priv_key)
btc_test_account

<PrivateKeyTestnet: mzdKk2y5V73b8Fnzzk7npFyzrD6YzsG7mB>

In [9]:
create_tx(BTCTEST,btc_test_account,"mzdKk2y5V73b8Fnzzk7npFyzrD6YzsG7mB", 0.1)

ValueError: Transactions must have at least one unspent.